In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
bakery1000 = pd.read_csv('../apriori/1000/1000-out1.csv',sep = '\n', header = None)
bakery1000Mtemp = bakery1000.iloc[:,0].values
bakery1000M = [list(map(int, x.split(','))) for x in bakery1000Mtemp]
for x in bakery1000M:
    del x[0]

In [3]:
goods = pd.read_csv('../apriori/goods.csv')
goods['Flavor'] = goods['Flavor'] + "-" + goods['Food']
goodsDict = dict(zip(goods['Id'], goods['Flavor']))

In [4]:
exampleData = pd.read_csv('../example/out1.csv', sep = '\n', header = None)
exampleDataTemp = exampleData.iloc[:,0].values
exampleDataM = [list(map(int, x.split(','))) for x in exampleDataTemp]
for x in exampleDataM:
    del x[0]

In [5]:
def support(item, baskets):
    top = sum([1 if item in basket else 0 for basket in baskets])
    bottom = len(baskets)
    return top*1.0/bottom

In [6]:
def getPairs(F, k):
    outList = []
    for first in F:
        for second in F:
            if len(first) == len(second) == k:
                outList.append([first, second, tuple(set(first) | set(second))])
    return outList 

In [7]:
def getSubsets(tmpTuple):
    size = len(tmpTuple) - 1
    return set(itertools.combinations(list(tmpTuple), size))

In [8]:
def candidate(F,k):
    C = set()
    for pair in getPairs(F,k):
        if len(pair[2]) == len(pair[0]) + 1:
            c = pair[2]
            flag = True
            for s in getSubsets(c):
                if s not in F:
                    flag = False
            if flag == True:
                if c not in C:
                    C.add(tuple(c))
    return C

In [9]:
def apriori(T, minSup):
    FullSupport = []
    I =  list(set(x for l in T for x in l))
    Fk = []; Fk.append(None);
    firstIter = set(tuple([x]) for x in filter(lambda a: a != -1, [x if support(x, T) >= minSup else -1 for x in I]))
    Fk.append(firstIter)
    k = 2
    while len(Fk[k-1]) != 0:
        print("running")
        Ck = candidate(Fk[k-1], k-1)
        count = {}
        for c in Ck:
            count[c] = 0
        for t in T:
            for c in Ck:
                if set(c) <= set(tuple(t)):
                    count[c] += 1
        tmpFk = []
        for c in Ck:
            if count[c]*1.0/len(T) >= minSup:
                tmpFk.append(c)
        Fk.append(set(tmpFk))
        newSet = set(x for x in Fk[k-1])
        for newTerm in Fk[k]:
            for oldTerm in Fk[k-1]:
                if set(oldTerm).issubset(newTerm) == True:
                    if oldTerm in newSet:
                        newSet.remove(oldTerm)
                        #print("After: ", newSet)
        Fk[k-1] = set(tuple(sorted(x)) for x in newSet)
        k += 1
    Fk = Fk[1:-1]
    return Fk
                    
            
        
    
    

In [10]:
def getCorrect():
    t = np.linspace(.05, .01, 20)
    for minS in t:
        ap = addNamesFreq(apriori(bakery1000M, minS),goodsDict)
        print("---- Apriori: ", minS, " ----")
        print("NumRules: ", len(genRules(bakery1000M, ap, .85)))
        for i, val in enumerate(ap):
            print(i+1, ": ", len(val))
            

In [11]:
def genRules(data, freqItemSet, minConf):
    allRules = []
    freqItemSet = freqItemSet[1:]
    freqItemSet = [item for sublist in freqItemSet for item in sublist]
    for itemSet in freqItemSet:
        for item in itemSet:
            noItem=set(itemSet)-(set([item]))
            item = set([item])
            conf = confidence(data,noItem,item)
            sup = supportOut(data,noItem,item)
            confItem = [noItem, item, conf, sup]
            if conf >= minConf:
                allRules.append(confItem)
                
            
    return allRules
                    

In [12]:
def confidence(T,set1,set2): #Dataset,leftSideList,rightSideList
    top = sum([1 if (set1|set2) <= set(basket) else 0 for basket in T])
    bottom = sum([1 if (set1) <= set(basket) else 0 for basket in T])
    if(bottom == 0 | top == 0):
        return 0
    return top*1.0/bottom

In [13]:
def supportOut(T, set1,set2):
    return sum([1 if (set1|set2) <= set(basket) else 0 for basket in T])/len(T)

In [14]:
tmpOut = apriori(bakery1000M, .01)

In [15]:
def getSupportFreq(T, output):
    allFreq = []
    for groupSize in output:
        for term in groupSize:
            termSup = supportOut(T, set(term), set([]))
            termOut = [term,termSup]
            allFreq.append(termOut)
    return allFreq
            
    

In [16]:
def addNamesFreq(output, goods):
    groupList=[]
    for i, basket in enumerate(output):
        basketList = []
        for value in list(basket[0]):
            basketList.append(goods[value])
        groupList.append([basketList, basket[1]])
    return groupList

In [17]:
def addNames(output, goods):
    ruleList = []
    for i, rule in enumerate(output):
        termList = []
        for j, terms in enumerate(rule[:-2]):
            setList = []
            for k, objectId in enumerate(list(terms)):
                setList.append(goods[objectId])
            termList.append(setList)
        termList.append(rule[2])
        termList.append(rule[3])
        ruleList.append(termList)
    return ruleList  

In [18]:
def printCorrectly(output):
    for i, rule in enumerate(output):
        print("Rule: ", i, '   ', ','.join(rule[0]), " --> ", 
                rule[1][0]) 
        print("              Support: ", str(rule[3]))
        print("              Confidence: ", str(rule[2]))
        

In [19]:
def findRules(T, minSup, minConf, goodsDictTmp, bakery = False):
    aprioriOutput = apriori(T, minSup)
    print("hi")
    aprioriNames = None
    if(bakery == True):
        aprioriNames = addNamesFreq(getSupportFreq(T, aprioriOutput), goodsDictTmp)
        for i, itemSet in enumerate(aprioriNames):
            print('Set # ', i+1, " ", itemSet)
    genRulesOutput = genRules(T, aprioriOutput, minConf)
    if(bakery == True):
        genRulesOutput = addNames(genRulesOutput, goodsDictTmp)
    print("**********************************************")
    printCorrectly(genRulesOutput)
    

In [20]:
bakery5000 = pd.read_csv('../apriori/5000/5000-out1.csv',sep = '\n', header = None)
bakery5000Mtemp = bakery5000.iloc[:,0].values
bakery5000M = [list(map(int, x.split(','))) for x in bakery5000Mtemp]
for x in bakery5000M:
    del x[0]

In [21]:
def findMinSupport(dataSet,goodsDict):
    t = np.linspace(.05, .01, 5)
    for minSup in t:
        print("******* ",minSup," *******")
        findRules(dataSet, minSup, .8, goodsDict, bakery=True)
    

In [22]:
bakery20000 = pd.read_csv('../apriori/20000/20000-out1.csv',sep = '\n', header = None)
bakery20000Mtemp = bakery20000.iloc[:,0].values
bakery20000M = [list(map(int, x.split(','))) for x in bakery20000Mtemp]
for x in bakery20000M:
    del x[0]

In [23]:
bakery75000 = pd.read_csv('../apriori/75000/75000-out1.csv',sep = '\n', header = None)
bakery75000Mtemp = bakery75000.iloc[:,0].values
bakery75000M = [list(map(int, x.split(','))) for x in bakery75000Mtemp]
for x in bakery75000M:
    del x[0]

In [24]:
bingoData = pd.read_csv('../apriori/bingoBaskets.csv',sep = '\n', header = None)
bingoData = bingoData.iloc[:,0].values
bingoData = [list(map(int, x.split(','))) for x in bingoData]
for x in bingoData:
    del x[0]

In [25]:
authors = pd.read_csv('../apriori/authors.psv',sep = '|', header = None)

In [26]:
#authors = authors[authors[1] != ' *N/A']
authorsDict = dict(zip(list(authors[0]), list(authors[1])))

In [27]:
#findRules(bingoData, .048, .8, authorsDict, bakery=True)

In [28]:
basketFactors = pd.read_csv('../example/factor_baskets_sparse.csv',sep = '\n', header = None)
#bingoData = bingoData.iloc[:,0].values
#bingoData = [list(map(int, x.split(','))) for x in bingoData]
#for x in bingoData:
 #   del x[0]

In [29]:
baskets = basketFactors.iloc[1:,0].values
baskets = [list(map(int, x.split(','))) for x in baskets]
for x in baskets:
    del x[0]

In [30]:
newBaskets = []
for x in baskets:
    t = x[::2]
    newBaskets.append(t)
    

In [31]:
factors = pd.read_csv('../example/factors.csv')

In [32]:
factorsDict = dict(zip(list(factors['tf_id']), list(factors['transfac'])))

In [33]:
tmpBaskets= newBaskets[0:10]